In [78]:
from selenium import webdriver
import time



# login screen move
def login_screen_url() :
    global driver
    driver = webdriver.Chrome('C:\/chromedriver.exe') # 제어하기 위해 변수 선언
    driver.maximize_window() ### 창의 해상도 크기 최대화 (전체화면) ###
    url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb" # 이동할 링크주소를 변수에 저장
    driver.get(url) # 해당 링크로 이동

    return driver



# user info (login)
def user_info():
    
    # ID
    xpath = "//*[@id='user_email']" 
    input_search = driver.find_element_by_xpath(xpath).send_keys("ID")


    # password
    xpath2 = "//*[@id='user_password']" 
    input_search = driver.find_element_by_xpath(xpath2).send_keys("password")


    # button click
    xpath3 = "//*[@id='signInSignInCon']/div[2]/div/section[2]/fieldset/button" # x 버튼, //button[@class='closeBtn']에다가 "" 로감싸야함
    login = driver.find_element_by_xpath(xpath3).click()


    
# Enterprise input
def Enterprise():
    # 기업명 이름
    xpath4 = "//*[@id='search_bar_search_query']" 
    Enterprise_input = driver.find_element_by_xpath(xpath4).send_keys("데이터 추출할 기업명 입력")
    
    # 검색 버튼 클릭
    search = "//*[@id='search_form']/div/button"
    btn_search = driver.find_element_by_xpath(search).click()

    # 검색한 기업 이름 클릭
    xpath5 = "//*[@id='mainContents']/div[1]/div/div[2]/div[1]/div[1]/a"
    Enterprise_name = driver.find_element_by_xpath(xpath5).click()
    time.sleep(2)
    
    
    # 검색한 기업의 면접 리뷰 클릭
    xpath6 = " //*[@id='viewCompaniesMenu']/ul/li[5]/a"
    review = driver.find_element_by_xpath(xpath6).click()
    time.sleep(2)
    
    scroll_down = driver.execute_script("window.scrollTo(0, 650)") 

    
    
    # 검색한 기업의 직종 select box 클릭
    xpath7 = "//*[@id='occupation_select']"
    job = driver.find_element_by_xpath(xpath7).click()
    time.sleep(2)
    
    # select box 를 클릭 해서 IT/인터넷 클릭 (Value는 source 로 open 후 확인)
    select_box = driver.find_element_by_xpath("//select/option[@value='11600']").click()
    time.sleep(2)

    
    
    # select box로 인해 스크롤 초기화 -> 다시 내려주기 
    scroll_down = driver.execute_script("window.scrollTo(0, 650)") 

    
    # 검색한 기업의 직급 select box 클릭
    xpath8 = "//*[@id='job_rank_select']"
    job_rank = driver.find_element_by_xpath(xpath8).click()
    time.sleep(2)
    
    # select box 를 클릭 해서 사원-대졸 클릭 (Value는 source 로 open 후 확인)
    select_box = driver.find_element_by_xpath("//select/option[@value='1']").click()
    time.sleep(2)
    
    
    # select box로 인해 스크롤 초기화 -> 다시 내려주기 
    scroll_down = driver.execute_script("window.scrollTo(0, 650)") 

    
    
# 본격적인 crawling
def craw() :
    data = []
    idx = 1

    print("★★★★★ 기출 면접 data crawling 시작합니다 ★★★★★")

    try : 
        
        # Pagination 1~99
        for j in range(1,100):
            url = f"https://www.jobplanet.co.kr/companies/23717/interviews_by_filter?by_occupation=11600&by_job_rank=1&by_success=&page={j}"
            driver.get(url)
            
                
        
            # 해당 page의 i번째 게시글 데이터 추출 
            for i in range(1,6) :
                theme = driver.find_element_by_xpath(f"//*[@id='viewInterviewsList']/div/div/section[{i}]/div/div[2]/div/div[1]/h2")    
                question = driver.find_element_by_xpath(f"//*[@id='viewInterviewsList']/div/div/section[{i}]/div/div[2]/div/dl/dd[1]/span")
                question2 = driver.find_element_by_xpath(f"//*[@id='viewInterviewsList']/div/div/section[{i}]/div/div[2]/div/dl/dd[2]")
                question_result = driver.find_element_by_xpath(f"//*[@id='viewInterviewsList']/div/div/section[{i}]/div/div[2]/div/div[2]/div/dl/dd[1]")



                # replace \n 제거
                res_theme = theme.text.replace("\n","")
                res_question = question.text.replace("\n","")
                res_question2 = question2.text.replace("\n","")
                res_question_result = question_result.text.replace("\n","")



                # result 결과
                data.append(res_theme)
                data.append(res_question)
                data.append(res_question2)
                data.append(res_question_result)


                print(f"{idx}번째 게시글")
                print()
                print("주제:",data[0])
                print()
                print("면접질문:",data[1])
                print()
                print("면접답변 혹은 면접느낌:",data[2])
                print()
                print("면접결과:",data[3])
                print()
                print("---------------------------------------------------")
                idx += 1
                data = [] # 출력했으니 초기화
            
            
    
    except Exception as e :
        print("게시글 데이터 추출 에러",e) 
    
# --------------------------- main -------------------------
login_screen_url() # jobplanet login screen url 
time.sleep(1)

user_info() # ID & PWD input -> login button click
time.sleep(2)

Enterprise() # 기업명 입력 후 면접 버튼 클릭 후 직군 , 직급 클릭 
craw() # 면접 review data 추출

★★★★★ 기출 면접 data crawling 시작합니다 ★★★★★
1번째 게시글

주제: "1차 면접은 인사팀과 실무 면접관 한 분씩 오셔서 2:1로 진행되었습니다.2차 면접은 CTO와 1:1 면접입니다."

면접질문: 1차 면접은 자기소개와 기술 관련 질문이 있었고 직무경험과 같은 질문이 있었습니다.2차 면접도 역시 기술면접과 자기소개 등 면접 질문이 있었습니다.

면접답변 혹은 면접느낌: 1차의 경우 자신이 지원한 직무의 기술을 잘 알고 있다면 크게 문제없이 답변할 수 있습니다.2차의 경우 기술면접도 중요하지만 왜 회사에서 지원자를 채용해야하는지(직무 경험 혹은 이에 대한 자신의 강점을 제시해야 될 것 같습니다.)

면접결과: 대기중

---------------------------------------------------
2번째 게시글

주제: "지원하고 3일인가 있다가 연락와서 면접 가능한 날짜 잡고 면접을 보러감긴장하지 않도록 잘 이끌어 주시고 끝나면 고생하셨다며 엘베까지 마중도 나와주셔서 기분 좋게 면접 봤던 기억이 있음"

면접질문: 1차 인성 면접은 1:1로 진행했고, 2차는 면접관 두분이 들어오셔서 1:2로 진행간단한 자기소개부터 시작해서 주로 이력서와 자기소개서에 썼던 내용들 위주로 질문함자신이 지원한 직무에 대한 이해는 필수

면접답변 혹은 면접느낌: 주로 이력서와 자기소개서를 보고 꼼꼼하게 질문 하셨음자신이 지원한 직무에 대한 이해를 기준으로 잘 정리해서 면접보면 될거같음

면접결과: 불합격

---------------------------------------------------
3번째 게시글

주제: "지원후 1주일 지나서 전화로 연락이 옴.면접 시간은 최대한 배려해 줬고 2일 후 면접 보러 감.많이 떨렸는데 면접 전 친절하게 안내해 줘서 좀 풀림."

면접질문: 면접관 한분이 들어와서 그동안 했던 프로젝트 등에 대해 질문.이력서에 있는 내용 위주로 질문.이후 회사설명 해주시고 1차 면접은 끝남.대체적으로 편안한 분위기에서 면

30번째 게시글

주제: "인사팀면접, 실무자면접, 과제, 임원면접으로 진행됨인사팀면접과 실무자면접은 동일한 날에 진행되었고, 임원면접전 과제가 있었음"

면접질문: 지금까지 경력 사항들에 대해서 전반적으로 설명해주세요.

면접답변 혹은 면접느낌: 전반적으로 경력사항들에 대해서 자세히 설명하였음

면접결과: 합격

---------------------------------------------------
31번째 게시글

주제: "1일차: 인사 - 기술 면접2일차: 임원 면접"

면접질문: 인사 면접에서 희망 연봉 물어봄. 연봉 안 맞으면 다음 면접 진행 불가연봉은 1/13

면접답변 혹은 면접느낌: 면접 분위기는 좋지만 연봉 제안에 어려움이 있다

면접결과: 불합격

---------------------------------------------------
32번째 게시글

주제: "이메일로 연락 및 면접을 진행하였습니다."

면접질문: 자기소개문제를 해결하는 접근법입사하고 나서 하고 싶은 일.

면접답변 혹은 면접느낌: 보통

면접결과: 불합격

---------------------------------------------------
33번째 게시글

주제: "인사팀 면접 후 실무자 면접 진행 했습니다.하루에 모든 면접이 다 이루어 졌습니다."

면접질문: 서울에 편의점이 몇 개나 있을 건 같은지 질문했습니다.맥켄지 입사질문으로 유명했던 질문이라대충 제방식대로 추론하는 법을 애기 했던 것 같습니다

면접답변 혹은 면접느낌: 대체로 무난한 질문을 받아 난해하지 않은 면접이었으나아직도 인사팀에서 실무관련 질문을 심도있게 물어봤던건인해가 가지 않네요

면접결과: 합격

---------------------------------------------------
34번째 게시글

주제: "면접은 평이한 수준이었으나 간혹 당황스러운 질문이 있었음. 화상 면접으로 진행."

면접질문: 지원 포지션과 관련된 경험. 전 직장에서 문제를 해결했던 예시. 카페24를 


주제: "하루에 인사팀 면접, 실무 면접 보았고 시간은 한시간 반 정도 소요된 것 같다"

면접질문: 인사팀 인성 면접은 난생 처음이라 당황스러웠고 준비를 제대로 못했다(인성이라는 게 준비되는 건 아니지만..) 그리고 실무 면접은 직무에 맞는 사람인지에 대해 여러 질문을 하였는데 관련 경력이 없어도 아는 게 많으면 어필이 될 수도 있었을 것 같다. 당연한 얘기지만, 자신만의 특별한 강점이 있어야 할 듯

면접답변 혹은 면접느낌: 질문에 맞는 답변을 하였는데 돌이켜보면 많이 부족했던 것 같고 관련 경력이 없어서 어필이 잘 되지 못했던 것 같다.

면접결과: 불합격

---------------------------------------------------
64번째 게시글

주제: "코딩테스트-직무면접-인성면접코딩테스트 제외 면접 결과는 대부분 일주일 이내로 발표남."

면접질문: 프론트/백 직무를 나누는 것에 대해 어떻게 생각하는지?데브옵스를 아는지?그 외 자신있는 언어에 대한 이론적 지식들

면접답변 혹은 면접느낌: 면접관이 친절했다.내 대답이 틀렸을 때 느낌이 왔다. 떨어질 것을

면접결과: 불합격

---------------------------------------------------
65번째 게시글

주제: "결과 통보가 매우 빨랐음 (이틀 뒤 통보)"

면접질문: 솔루션이 무엇인지 알고있는지?지원동기가 무엇인지?공백기간동안 무엇을 했는지?

면접답변 혹은 면접느낌: 생각보다 면접 분위기가 무겁지 않았음

면접결과: 합격

---------------------------------------------------
66번째 게시글

주제: "인사팀 분께서 반갑게 맞아주시고 면접과 관련된 사항을 안내해 주십니다. 이후 개발자 두분이 오셔서 여러가지 질문을 하셨고. 간단한 코딩문제 및 객관식 문제를 풀었습니다."

면접질문: 쇼핑몰과 관련된 부서였는데 주로 대용량처리에 관련한 질문이 많았습니다. 대규모 트래픽 처리를 위한 개발자로서 코딩이나 인프라처

94번째 게시글

주제: "포지션에 따라 다름. 실무진 면접->인사팀 면접 순으로 진행 됨.합격 통보는 매우 면접 후 3일 내에 받음.이력서 접수순서로 면접보고 채용시 마감하기 때문에 이력서는 빨리 제출할수록 좋음"

면접질문: 기존에 받았던 연봉보다 적은데 상관없는지?입사동기가 무엇인지?지원한 직무랑 다른 직무로 채용하면 입사의향이 있는지?

면접답변 혹은 면접느낌: 사실 실무적인 부분보다는 인성적인 부분에 치중된 면접이어서 견본이 될만한 답변이 없습니다.

면접결과: 합격

---------------------------------------------------
95번째 게시글

주제: "코딩테스트 합격 후에 전화로 면접 날짜를 잡음.면접은 2대 1 면접이었고, 기술면접과 인성면접을 같이 봄.편안한 분위기였고 질문에 대한 답변을 모를 시 힌트를 주기도 함."

면접질문: 시큐어 코딩에 관련된 DB 스킬에 대한 질문이 있었음.

면접답변 혹은 면접느낌: 시큐어 코딩에 대해 부족함이 있어 모른다고 답변함.

면접결과: 불합격

---------------------------------------------------
96번째 게시글

주제: "하루에 1차 인사팀 인성면접, 2차 실무진 직무면접 진행"

면접질문: 직무에서 하는 일이 뭐라고 생각하는지, 이전 직장에서 어려웠던 업무, 맡았던 업무 구체적으로 말하고 성과는 어땠는지 , 만약에 상사가 업무와 상관없는 일을 시킨다면? 성과지향인지 과정지향인지

면접답변 혹은 면접느낌: 자소서에 기반으로 질문, 구체적인 답변 원함, 전반적으로 딱딱한 분위기.

면접결과: 대기중

---------------------------------------------------
97번째 게시글

주제: "서류합격하고 코딩테스트를 보러갔는데 생각보다 사람이 많아서 놀랐고 무슨 학원? 같은데서 봤음. 생각한거 보다 시간이 걸리는 문제들이라서 망함"

면접질문: 인터페이스 구현 하는거, 지문 보고 간단함 게임 만드는 것 등등


121번째 게시글

주제: "지원 후 3일안에 연락이 와서 그 다음주에 면접일정을 잡음인사팀면접은 최대한 편안한 분위기를 만들어 주려고 배려하는 것을 느낌.기술면접도 편안한 분위기를 만들어 주려는 듯하였다."

면접질문: 자신이 남들보다 잘알고있는 것을 설명하시오.

면접답변 혹은 면접느낌: 제대로 대답하지 못함.

면접결과: 불합격

---------------------------------------------------
122번째 게시글

주제: "HR에서 연락이 와서 면접보러감HR면접은 아주 좋앗음 상냥하고 친절하게 물어보심가치관이나 어떤 인생을 살앗는지에 대한 전반적 사항을 물어봄솔직하게 답변햇음문제는 기술면접실무자랑 이사급? 인지모르겟지만 두분이 들어옴HR에서 분명 자세히 적던데 하나도 안읽어보고온듯함같은 질문을 계속 답한듯한 느낌뭘 아는지 얼마나 하는지 이런거 안물어봄면접보실 준비를 안하신것 같음..지원자만 면접준비하는 것이 아닌거 같은데.."

면접질문: 인생에서 제일 좋은 경험은? 인성관련질문이 좋았음 다양한 인생 질문을 해줌

면접답변 혹은 면접느낌: 10일 후

면접결과: 대기중

---------------------------------------------------
123번째 게시글

주제: "인사팀에서 사람인 이력서를 써칭해서 연락이 왔고, 면접을 보게 되었습니다. 면접관은 여자 두명이었습니다. 합격해도 오기 싫을 정도로 면접자에 대한 배려가 없었습니다.면접 질문에 대한 답을 하면 "아닐거 같은데~평소에 긴장하지 않나요" 이런식으로 비아냥거렸습니다. 모든 대답에 면접관 둘이서 대놓고 비웃는 태도를 보였습니다. 면접 질문은 자기소개. 지원동기. 경력사항. 장단점에 대한 아주 기초적인 질문이었습니다. 어렵거나 특이한 질문은 하나도 없었습니다.신규 인력을 까다롭게 뽑기 이 전에, 기존 인력에 대한 검토가 우선이라고 생각합니다.^ ^"

면접질문: 자기소개를 해보세요. 장단점은 뮈에요? 지원동기있어요?

면접답변 혹은 면접느낌: ^^^^